In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import evaluate
from utils import save_response

# Llama 3 - 8B Instruct Model

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
messages = [
    {"role": "system", "content": f"""You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style."""},
    {"role": "user", "content": f"""Can you help me write a story with the following details:
Prompt: A action packed adventure story based off of Indiana Jones, include outrunning deadly traps, a chase scene with a treasure hunter and a villain, jungles and ancient ruins.
Distance: 13.1 miles
Average Walking Speed: 2.5 mph
Average Running Speed: 7.2 mph
Training Style: Half Marathon"""},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [4]:
hyperparams = {
    "max_new_tokens": 750, 
    "temperature":0.6, 
    "top_p":0.9, 
    "do_sample": True
}
outputs = model.generate(
    input_ids,
    max_new_tokens=hyperparams["max_new_tokens"],
    eos_token_id=terminators,
    do_sample=hyperparams["do_sample"],
    temperature=hyperparams["temperature"],
    top_p=hyperparams["top_p"],
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\colin\OneDrive\Desktop\Applied ML\NLP Assignment\Natural_Language_Assignment\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
response = outputs[0][input_ids.shape[-1]:]
output = tokenizer.decode(response, skip_special_tokens=True)
print(output)

Here's a story for you:

**The Quest for the Golden Idol**

As I ventured deeper into the dense jungle, the air grew thick with humidity and the sounds of exotic birds and animals echoed through the trees. I had been searching for weeks, following cryptic clues and ancient maps, to uncover the location of the legendary Golden Idol. Rumors whispered that it was hidden within the ruins of a long-lost civilization, guarded by deadly traps and treacherous terrain.

As I pushed through the underbrush, I stumbled upon a clearing, and my eyes widened in awe. Before me lay the entrance to the ancient city, a sprawling complex of crumbling temples and pyramids. I could feel the excitement coursing through my veins as I approached the entrance.

Suddenly, I heard the sound of rustling leaves behind me. I turned to see a rival treasure hunter, the notorious Dr. Vortex, emerging from the jungle. His eyes locked onto mine, and a sinister grin spread across his face.

"You'll never find the Golden I

In [6]:
#save_response("LLAMA3-8B-INSTRUCT", hyperparams, output)

In [7]:
# perplexity = evaluate.load("perplexity", module_type="metric")
# input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
# results = perplexity.compute(model_id=model_id,
#                              add_start_token=False,
#                              predictions=input_texts, 
#                              device="cuda")
# print(list(results.keys()))

In [8]:
# print(round(results["mean_perplexity"], 2))